In [43]:
import eclipsr as ecl
from astroquery.mast import Observations, Catalogs
import eleanor
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
import time
import astropy.units as u

target_data = eleanor.TargetData(eleanor.Source(fn='hlsp_gsfc-eleanor-lite_tess_ffi_s0003-0000000' + str(514257394) + '_tess_v1.0_lc.fits', fn_dir='E:\Coding\PyCharm Python Projects\light curve testing 2\mastDownload\HLSP\hlsp_gsfc-eleanor-lite_tess_ffi_s0003-0000000' + str(514257394) + '_tess_v1.0_lc'))
lc = target_data.to_lightkurve()

times = lc.time.value
signal = lc.flux.value

times, signal = ecl.utility.ingest_signal(times, signal, tess_sectors=False)

No postcard object will be created for this target.


In [44]:
start_time = time.time()
t_0, period, score, sine_like, wide, n_kernel, width_stats, depth_stats = ecl.find_eclipses(times, signal, mode=1, tess_sectors=False)
end_time = time.time()
ecl_time = end_time - start_time

period, ecl_time

(3.42460031603206, 0.14103150367736816)

In [45]:
T = np.linspace(1, 27, 10000)

start_time = time.time()
bls = lc.to_periodogram(method='bls', period=T, frequency_factor=500)
end_time = time.time()
bls_time = end_time - start_time

bls_period = float(bls.period_at_max_power / u.d)
bls_period, bls_time

(1.7098709870987097, 0.6470627784729004)

In [46]:
mean = np.mean(signal)
std = np.std(signal)

z_scores = [(x - mean) / std for x in signal]

np.array(z_scores).min(), np.array(z_scores).max()

(-4.055011947596159, 1.859607759643517)

In [47]:
mean = np.mean(signal)
var = np.var(signal)

n_signal = signal - mean

acf = np.correlate(n_signal, n_signal, mode='full')[-len(n_signal):]
# acf = acf / var

# acf.min(), acf.max()
acf

array([ 7.47438669e-02,  6.82605329e-02,  6.19154249e-02,  5.49208747e-02,
        5.02169856e-02,  4.71852213e-02,  4.51794726e-02,  4.35774762e-02,
        4.22189702e-02,  4.10981531e-02,  3.98010912e-02,  3.86670925e-02,
        3.74178748e-02,  3.61400045e-02,  3.42907157e-02,  3.26527705e-02,
        3.08032211e-02,  2.93852189e-02,  2.74651521e-02,  2.52692508e-02,
        2.22590655e-02,  2.02383324e-02,  1.84164144e-02,  1.67508971e-02,
        1.46657858e-02,  1.23169338e-02,  1.05743806e-02,  8.45483652e-03,
        7.12941872e-03,  5.09071720e-03,  3.48653063e-03,  1.28876625e-03,
       -4.65506527e-04, -2.80079335e-03, -4.52813850e-03, -6.53297429e-03,
       -7.84671959e-03, -9.29670498e-03, -1.06791132e-02, -1.20370317e-02,
       -1.33851440e-02, -1.45655575e-02, -1.58172498e-02, -1.65911354e-02,
       -1.80741485e-02, -1.91128071e-02, -2.01011909e-02, -2.08995743e-02,
       -2.18380804e-02, -2.28940831e-02, -2.40071454e-02, -2.50843129e-02,
       -2.61606514e-02, -

In [48]:
inflection = np.diff(np.sign(np.diff(acf)))
peaks = (inflection < 0).nonzero()[0] + 1
delay = peaks[acf[peaks].argmax()]

delay

147

In [49]:
# percent difference between period / 2 and bls_period
abs((period / 2 - bls_period)) / ((bls_period + period / 2) / 2)

0.0014196665299969183

In [50]:
period / 2, bls_period

(1.71230015801603, 1.7098709870987097)